<a href="https://colab.research.google.com/github/ganeshmp01/DS-Classwork/blob/main/text_classification_movie_reviews_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [41]:
import pandas as pd
import numpy as np
import contractions
import unicodedata
from bs4 import BeautifulSoup
import re
import tqdm


In [32]:
dataset = pd.read_csv("/content/movie_reviews.csv.bz2")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [33]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [34]:
def strip_html_tags(text):
  soup = BeautifulSoup(text,'html.parser')
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+','\n',stripped_text)
  return stripped_text

In [35]:
def remove_accented_characters(text):
  text = unicodedata.normalize("NFKD",text).encode("ascii",'ignore').decode('utf-8','ignore')
  return text

In [42]:
def pre_process_corpus(texts):
  preprocessed_text = []
  for doc in tqdm.tqdm(texts):
    doc = strip_html_tags(doc)
    #doc = doc.translate(doc.maketrans("\n\t\r",'   '))
    doc = doc.lower()
    doc = remove_accented_characters(doc)
    doc = contractions.fix(doc)
    doc = re.sub(r'[^a-zA-Z0-9\s]','',doc,re.I|re.A)
    doc = re.sub(r' +',' ',doc)
    doc = doc.strip()
    #doc = "".join(doc)
    preprocessed_text.append(doc)
  return preprocessed_text

In [43]:
reviews  = dataset['review'].values
sentiment = dataset['sentiment'].values

train_reviews = reviews[:35000]
train_sentiment = sentiment[:35000]

test_reviews = reviews[35000:]
test_sentiment = sentiment[35000:]

In [44]:
%%time
norm_train_reviews = pre_process_corpus(train_reviews)
norm_test_reviews = pre_process_corpus(test_reviews)

  2%|▏         | 746/35000 [00:00<00:37, 916.04it/s]<ipython-input-34-46650b581e77>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text,'html.parser')
100%|██████████| 15000/15000 [00:08<00:00, 1813.90it/s]

CPU times: user 27.1 s, sys: 253 ms, total: 27.3 s
Wall time: 28.5 s


In [45]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer(binary = True,ngram_range=(1,2))

In [46]:
cv_train_features = cv.fit_transform(norm_train_reviews)

In [47]:
cv.get_feature_names_out()

array(['00', '00 agent', '00 for', ..., 'zzzzzzzzzzzzzzzzzz if',
       'zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz',
       'zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz 8ooops'], dtype=object)

In [48]:
cv_test_features = cv.transform(norm_test_reviews)

In [50]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators = 200,max_depth=2)

rfc.fit(cv_train_features,train_sentiment)

rfc_bow_predictions = rfc.predict(cv_test_features)

In [51]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

labels = ['negative','positive']
print(classification_report(test_sentiment,rfc_bow_predictions))
pd.DataFrame(confusion_matrix(test_sentiment, rfc_bow_predictions),index = labels,columns = labels)

              precision    recall  f1-score   support

    negative       0.80      0.76      0.78      7490
    positive       0.78      0.81      0.79      7510

    accuracy                           0.79     15000
   macro avg       0.79      0.79      0.79     15000
weighted avg       0.79      0.79      0.79     15000



,negative,positive
negative,5727,1763
positive,1434,6076


In [52]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Tv = TfidfVectorizer(use_idf= True,ngram_range=(1,2))

Tv_train_features = Tv.fit_transform(norm_train_reviews)
Tv_test_features = Tv.transform(norm_test_reviews)

In [53]:
rfc = RandomForestClassifier(n_estimators = 200,max_depth=2)

rfc.fit(Tv_train_features,train_sentiment)

rfc_tfidf_predictions = rfc.predict(Tv_test_features)

In [54]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

labels = ['negative','positive']
print(classification_report(test_sentiment,rfc_tfidf_predictions))
pd.DataFrame(confusion_matrix(test_sentiment, rfc_tfidf_predictions),index = labels,columns = labels)

              precision    recall  f1-score   support

    negative       0.80      0.75      0.78      7490
    positive       0.77      0.81      0.79      7510

    accuracy                           0.78     15000
   macro avg       0.78      0.78      0.78     15000
weighted avg       0.78      0.78      0.78     15000



,negative,positive
negative,5643,1847
positive,1422,6088
